<a href="https://colab.research.google.com/github/juanfer97/juan-rojas-entrega1/blob/main/03%20-%20modelo%20%20con%20GradientBoostingClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importa librerías básicas para manejo de datos y operaciones

import numpy as np
import pandas as pd
import os
import random

In [4]:
# Configura el entorno para Kaggle y descarga el dataset de la competencia
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
# Descomprime el archivo descargado del dataset
!unzip udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip

Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [6]:
#  Carga los archivos CSV de train y test en dataframes de pandas
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [7]:
#  Muestra las columnas del dataframe de entrenamiento
df_train.columns

Index(['ID', 'PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO',
       'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA',
       'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE',
       'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL', 'E_PRIVADO_LIBERTAD',
       'E_PAGOMATRICULAPROPIO', 'F_TIENECOMPUTADOR', 'F_TIENEINTERNET.1',
       'F_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL', 'INDICADOR_1', 'INDICADOR_2',
       'INDICADOR_3', 'INDICADOR_4'],
      dtype='object')

In [8]:
# Instala la librería unidecode para normalización de texto
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.6 MB/s eta 0:00:00


In [45]:
 # Normaliza el nombre del programa académico eliminando tildes y poniendo en mayúsculas
import unidecode
def estu_program_normalizado(df):

  def fillna(row):
    if pd.isna(row):
      return row
    row = row.upper().strip()
    row = unidecode.unidecode(row)
    row = ' '.join(row.split())
    return row

  df['E_PRGM_ACADEMICO'] = df['E_PRGM_ACADEMICO'].apply(fillna)


In [11]:
#  Muestra los valores únicos de la columna de programa académico
df_train['E_PRGM_ACADEMICO'].unique()

array(['ENFERMERIA', 'DERECHO', 'MERCADEO Y PUBLICIDAD',
       'ADMINISTRACION DE EMPRESAS', 'PSICOLOGIA', 'MEDICINA VETERINARIA',
       'INGENIERIA MECANICA', 'ADMINISTRACIÓN EN SALUD OCUPACIONAL',
       'INGENIERIA INDUSTRIAL', 'ADMINISTRACIÓN FINANCIERA',
       'HOTELERIA Y TURISMO', 'LICENCIATURA EN CIENCIAS SOCIALES',
       'LICENCIATURA EN PEDAGOGIA INFANTIL', 'COMUNICACION SOCIAL',
       'CIENCIA POLITICA',
       'PROFESIONAL EN GESTIÓN DE LA SEGURIDAD Y LA SALUD LABORAL',
       'MAESTRO EN MÚSICA', 'INGENIERIA MECATRONICA', 'TRABAJO SOCIAL',
       'LICENCIATURA EN BIOLOGIA Y EDUCACION AMBIENTAL',
       'INGENIERIA CIVIL', 'CONTADURIA PÚBLICA',
       'ADMINISTRACION EN SALUD', 'ADMINISTRACIÓN DE EMPRESAS',
       'ESTADISTICA', 'LICENCIATURA EN BIOLOGIA',
       'INGENIERIA AGROINDUSTRIAL', 'ZOOTECNIA',
       'COMUNICACION AUDIOVISUAL',
       'LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN HUMANIDADES-INGLES',
       'COMUNICACION SOCIAL  - PERIODISMO',
       'SEG

In [12]:
# Muestra la cantidad de valores nulos por columna (solo las que tienen nulos)
p = df_train.isna().sum()
p[p!=0]

,0
E_VALORMATRICULAUNIVERSIDAD,6287
E_HORASSEMANATRABAJA,30857
F_ESTRATOVIVIENDA,32137
F_TIENEINTERNET,26629
F_EDUCACIONPADRE,23178
F_TIENELAVADORA,39773
F_TIENEAUTOMOVIL,43623
E_PAGOMATRICULAPROPIO,6498
F_TIENECOMPUTADOR,38103
F_TIENEINTERNET.1,26629


In [15]:
# Muestra los valores únicos del departamento del programa y define función de normalización
programas = df_train['E_PRGM_DEPARTAMENTO'].unique()
programas

def estu_program_departamento(df):

  department_name = {
      'BOGOTÁ': 'BOGOTA',
      'NARIÑO': 'NARINO'
  }

  df['E_PRGM_DEPARTAMENTO'] = df['E_PRGM_DEPARTAMENTO'].str.strip()
  df['E_PRGM_DEPARTAMENTO'] = df['E_PRGM_DEPARTAMENTO'].replace(department_name)

In [16]:
# Muestra los valores únicos de estrato y define función para imputar nulos con la moda
estrato = df_train['F_ESTRATOVIVIENDA'].unique()
estrato

def fami_estrato(df):

  #Ej: Moda = Estrato 2
  moda = df['F_ESTRATOVIVIENDA'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].apply(fillna)

In [17]:
#  Muestra la columna de horas semanales de trabajo
df_train['E_HORASSEMANATRABAJA']

,E_HORASSEMANATRABAJA
0,Menos de 10 horas
1,0
2,Más de 30 horas
3,0
4,Entre 21 y 30 horas
...,...
692495,Entre 11 y 20 horas
692496,Más de 30 horas
692497,Menos de 10 horas
692498,Menos de 10 horas


In [18]:
#  Imputa valores nulos en horas semanales de trabajo con la moda
def estu_hora_trabaja(df):

  moda = df['E_HORASSEMANATRABAJA'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].apply(fillna)

In [19]:
#  Imputa valores nulos en valor de matrícula con la moda
def matricula_valor(df):

  moda = df['E_VALORMATRICULAUNIVERSIDAD'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].apply(fillna)

In [20]:
#  Muestra la moda de estrato de vivienda
df_train['F_ESTRATOVIVIENDA'].mode()[0]

'Estrato 2'

In [21]:
# Muestra los valores únicos de si tiene internet en casa

df_train['F_TIENEINTERNET'].unique() #Sale por probabilidad

array(['Si', 'No', nan], dtype=object)

In [22]:
# Muestra los valores únicos de educación del padre
df_train['F_EDUCACIONPADRE'].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [23]:
#  Muestra los valores únicos de educación del padre y define función para imputar nulos con la moda
df_train['F_EDUCACIONPADRE'].unique() #Hay nan

def fami_educacion_padre(df):

  moda = df['F_EDUCACIONPADRE'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].apply(fillna)

In [24]:
#  Muestra los valores únicos de si tiene lavadora y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENELAVADORA'].unique() #Hay nan

def fami_tienelavadora(df):

  percent_si = np.round(df['F_TIENELAVADORA'].value_counts(normalize=True).get('Si', 0), 4)

  def fillna(row):
    estrato = row['F_ESTRATOVIVIENDA']
    if pd.isna(row['F_TIENELAVADORA']):
      if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
          return 'Si'
      elif random.random() <= percent_si:
          return 'Si'
      else:
          return 'No'
    else:
      return row['F_TIENELAVADORA']
  df['F_TIENELAVADORA'] = df.apply(fillna, axis=1)

In [25]:
#  Calcula el porcentaje de 'Si' en la columna de lavadora
percent_si = np.round(df_train['F_TIENELAVADORA'].value_counts(normalize=True).get('Si', 0), 4)
percent_si

np.float64(0.8631)

In [26]:
#  Muestra los valores únicos de si tiene internet y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENEINTERNET'].unique()  # Hay nan

def fami_tieneinternet(df):
    percent_si = np.round(df['F_TIENEINTERNET'].value_counts(normalize=True).get('Si', 0), 4)

    def fillna(row):
        estrato = row['F_ESTRATOVIVIENDA']
        if pd.isna(row['F_TIENEINTERNET']):
            if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
                return 'Si'
            elif random.random() <= percent_si:
                return 'Si'
            else:
                return 'No'
        else:
            return row['F_TIENEINTERNET']

    df['F_TIENEINTERNET'] = df.apply(fillna, axis=1)


In [27]:
#  Muestra los valores únicos de si tiene internet (columna duplicada) y define función para imputar nulos
df_train['F_TIENEINTERNET.1'].unique()  # Hay nan

def fami_tieneinternet_1(df):
    percent_si = np.round(df['F_TIENEINTERNET.1'].value_counts(normalize=True).get('Si', 0), 4)

    def fillna(row):
        estrato = row['F_ESTRATOVIVIENDA']
        if pd.isna(row['F_TIENEINTERNET.1']):
            if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
                return 'Si'
            elif random.random() <= percent_si:
                return 'Si'
            else:
                return 'No'
        else:
            return row['F_TIENEINTERNET.1']

    df['F_TIENEINTERNET.1'] = df.apply(fillna, axis=1)


In [28]:
#  Calcula el porcentaje de 'Si' en la columna de internet
percent_si = np.round(df_train['F_TIENEINTERNET'].value_counts(normalize=True).get('Si', 0), 4)
percent_si

np.float64(0.8898)

In [29]:
#  Muestra los valores únicos de si tiene automóvil y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENEAUTOMOVIL'].unique()  # Hay nan

def fami_tieneautomovil(df):
    percent_si = np.round(df['F_TIENEAUTOMOVIL'].value_counts(normalize=True).get('Si', 0), 4)

    def fillna(row):
        estrato = row['F_ESTRATOVIVIENDA']
        if pd.isna(row['F_TIENEAUTOMOVIL']):
            if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
                return 'Si'
            elif random.random() <= percent_si:
                return 'Si'
            else:
                return 'No'
        else:
            return row['F_TIENEAUTOMOVIL']

    df['F_TIENEAUTOMOVIL'] = df.apply(fillna, axis=1)


In [30]:
#  Calcula el porcentaje de 'Si' en la columna de automóvil
percent_si = np.round(df_train['F_TIENEAUTOMOVIL'].value_counts(normalize=True).get('Si', 0), 4)
percent_si

np.float64(0.3641)

In [31]:
#  Muestra los valores únicos
df_train['E_PRIVADO_LIBERTAD'].unique()

array(['N', 'S'], dtype=object)

In [32]:
#  Muestra los valores únicos de si paga matrícula propia
df_train['E_PAGOMATRICULAPROPIO'].unique()

array(['No', 'Si', nan], dtype=object)

In [33]:
# Celda: Imputa valores nulos en pago de matrícula propia con la moda
def estu_pagamatricula(df):

  moda = df['E_PAGOMATRICULAPROPIO'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['E_PAGOMATRICULAPROPIO'] = df['E_PAGOMATRICULAPROPIO'].apply(fillna)

In [34]:
#  Muestra los valores únicos de si tiene computador y define función para imputar nulos según estrato y probabilidad
df_train['F_TIENECOMPUTADOR'].unique() #Hay nan

def fami_tienecomputador(df):

  percent_si = np.round(df['F_TIENECOMPUTADOR'].value_counts(normalize=True).get('Si', 0), 4)

  def fillna(row):
    estrato = row['F_ESTRATOVIVIENDA']
    if pd.isna(row['F_TIENECOMPUTADOR']):
      if ((estrato != 'Sin estrato') or (estrato != 'Estrato 1') or (estrato != 'Estrato 2')):
          return 'Si'
      elif random.random() <= percent_si:
          return 'Si'
      else:
          return 'No'
    else:
      return row['F_TIENECOMPUTADOR']
  df['F_TIENECOMPUTADOR'] = df.apply(fillna, axis=1)

In [37]:
#  Muestra los valores únicos de la columna FAMI_TIENEINTERNET.1
df_train['F_TIENEINTERNET.1'].unique()

array(['Si', 'No', nan], dtype=object)

In [39]:
#  Muestra los valores únicos de educación de la madre
df_train['F_EDUCACIONMADRE'].unique()

array(['Postgrado', 'Técnica o tecnológica incompleta',
       'Secundaria (Bachillerato) completa', 'Primaria completa',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) incompleta',
       'Educación profesional incompleta',
       'Educación profesional completa', 'Primaria incompleta', nan,
       'Ninguno', 'No Aplica', 'No sabe'], dtype=object)

In [40]:
#  Imputa valores nulos en educación de la madre con la moda
def fami_educacion_madre(df):

  moda = df['F_EDUCACIONMADRE'].mode()[0]

  def fillna(row):
    if pd.isna(row):
      return moda
    return row
  df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].apply(fillna)

In [41]:
#  Muestra los valores únicos de rendimiento global
df_train['RENDIMIENTO_GLOBAL'].unique()

array(['medio-alto', 'bajo', 'alto', 'medio-bajo'], dtype=object)

In [42]:
#  Convierte los valores de rendimiento global a valores numéricos
def rendimiento(df):

  rendimiento_replace = {
      'bajo': 0,
      'medio-bajo': 1,
      'medio-alto': 2,
      'alto': 3
  }

  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento_replace)


In [43]:
#  Aplica todas las funciones de limpieza y transformación al dataset
def limpiar_dataset_completo(df, isrendimiento=False):

    if not isrendimiento:
      rendimiento(df)
    estu_program_normalizado(df)
    estu_program_departamento(df)
    fami_estrato(df)
    fami_educacion_padre(df)
    fami_educacion_madre(df)
    fami_tienelavadora(df)
    fami_tieneinternet(df)
    fami_tieneinternet_1(df)
    fami_tieneautomovil(df)
    fami_tienecomputador(df)
    estu_pagamatricula(df)
    estu_hora_trabaja(df)
    matricula_valor(df)

    return df


In [46]:
#  Limpia el dataset de entrenamiento
df_train_limpio = limpiar_dataset_completo(df_train)

In [47]:
#  Limpia el dataset de test (sin transformar rendimiento)
df_test_limpio = limpiar_dataset_completo(df_test, isrendimiento=True)

In [49]:
#  Elimina la columna FAMI_TIENEINTERNET.1 del test limpio
df_test_limpio.drop(columns=['F_TIENEINTERNET.1'], inplace=True, axis=1)

In [50]:
# Convierte los valores numéricos de rendimiento global a texto
def rendimiento_back(df):

  rendimiento_replace = {
      0: 'bajo',
      1: 'medio-bajo',
      2: 'medio-alto',
      3: 'alto'
  }

  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].replace(rendimiento_replace)


In [51]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# 1. PREPARAR DATOS
data = df_train_limpio.copy()
x = data.drop(["RENDIMIENTO_GLOBAL", "ID"], axis=1)
y = data['RENDIMIENTO_GLOBAL']

cat_cols = x.select_dtypes(include=['object']).columns.tolist()

# 2. APLICAR LABEL ENCODING Y GUARDAR ENCODERS
label_encoders = {}
x_encoded = x.copy()

for col in cat_cols:
    le = LabelEncoder()
    x_encoded[col] = x_encoded[col].astype(str)
    x_encoded[col] = le.fit_transform(x_encoded[col])
    label_encoders[col] = le  # GUARDAR el encoder

# 3. SPLIT CON ESTRATIFICACIÓN
x_train, x_val, y_train, y_val = train_test_split(
    x_encoded, y, test_size=0.2, random_state=42, stratify=y
)


In [52]:
# Crea y entrena el modelo GradientBoostingClassifier con los parámetros optimizados

from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(
    n_estimators=1000,
    learning_rate=0.6,
    max_depth=3,
    random_state=42,
    max_features='sqrt',
    min_samples_leaf=15,
    min_samples_split=10,
)

model.fit(x_train, y_train)

GradientBoostingClassifier(learning_rate=0.6, max_features='sqrt',
                           min_samples_leaf=15, min_samples_split=10,
                           n_estimators=1000, random_state=42)

In [53]:
# Realiza predicciones sobre el conjunto de validación y calcula su precisión
predict_y = model.predict(x_val)
acc = accuracy_score(y_val, predict_y)
print(acc)

0.42940072202166063


In [56]:
test = df_test_limpio.copy()
y = test['ID']
test.drop(columns=['ID'], inplace=True, axis=1)

# Re-adding F_TIENEINTERNET.1 to match training features
test['F_TIENEINTERNET.1'] = test['F_TIENEINTERNET']

for col in cat_cols:
    if col in test.columns:
        test[col] = test[col].astype(str)

        # Manejar valores no vistos en entrenamiento
        le = label_encoders[col]

        def safe_transform(value):
            try:
                return le.transform([value])[0]
            except ValueError:
                # Si el valor no existe, usar el más frecuente del training
                return le.transform([le.classes_[0]])[0]

        test[col] = test[col].apply(safe_transform)

# Ensure the order of columns in 'test' matches 'x_encoded'
test = test[x_encoded.columns] # Reorder columns here

predecir = model.predict(test)

submit_df= pd.DataFrame({
    'ID': y,
    'RENDIMIENTO_GLOBAL': predecir
})

rendimiento_back(submit_df)

In [57]:
submit_df.to_csv('submission.csv', index=False)